In [1]:
import numpy as np
from astropy.io import fits

from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.fft import fft

import platform, multiprocessing
from multiprocessing import Pool

import copy

import Simpler_Classifier

from importlib import reload 


[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


my_module = reload(Simpler_Classifier) # reload of Simpler_Classifier without restart

In [2]:
HEADER_PATH = '/Users/runquanguan/Documents/dapall_excluded.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['plateifu']


In [3]:
if __name__ == '__main__':
    if platform.system() == "Darwin":
        multiprocessing.set_start_method('spawn')
        pool = multiprocessing.Pool(processes=10)
        results = []
        for result in tqdm(pool.imap_unordered(Simpler_Classifier.ring_test, plateifu), total = len(plateifu)):
            results.append(result)
        pool.close()
        pool.join() 


100%|██████████| 10737/10737 [2:38:14<00:00,  1.13it/s] 


In [4]:
result_copy = copy.deepcopy(results)

In [5]:
clean = []
for i in result_copy:
    if i != None:
        clean.append(i[0])
len(clean)


10632

In [6]:
store = []
for i in result_copy:
    if i != None:
        store.append(i)

In [7]:
miss_list = []
for data in list(plateifu):
    if data in clean:
        pass
    else:
        miss_list.append(data)

len(miss_list)

105

In [8]:
miss_results = []
for i in tqdm(miss_list):
    result = Simpler_Classifier.ring_test(i)
    miss_results.append(result)

100%|██████████| 105/105 [10:20<00:00,  5.91s/it]


In [9]:
clean_2 = []
for i in miss_results:
    if i != None:
        clean_2.append(i[0])
len(clean_2)

59

In [10]:
for i in miss_results:
    if i != None:
        store.append(i)
len(store)

10691

In [11]:
miss_list_2 = []
for i in miss_list:
    if i in clean_2:
        pass
    else:
        miss_list_2.append(i)
len(miss_list_2)

46

In [12]:
miss_results_2 = []
for i in tqdm(miss_list_2):
    result = Simpler_Classifier.ring_test(i)
    miss_results_2.append(result)

100%|██████████| 46/46 [03:47<00:00,  4.94s/it]


In [13]:
for i in miss_results_2:
    store.append(i)
    
len(store)

10737

In [14]:
plate_ifu = []
check = []
re_max = []



for i in store:
    if i != None:
        plate_ifu.append(i[0])
        check.append(i[1])
        re_max.append(i[2])

        
    else:
        pass

        

In [15]:
if len(store) == 10737:

    c1 = fits.Column(name='plateifu', array=np.array(plate_ifu), format='20A')
    c2 = fits.Column(name='check_percent', array=np.array(check), format='E')
    c3 = fits.Column(name='re_max', array=np.array(re_max), format='E')


    t = fits.BinTableHDU.from_columns([c1, c2, c3])
    t.writeto('dapall_ring_check_v3.fits')
else:
    pass

In [16]:
len(store)

10737